[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/IMF.ipynb)

# Implicit Matrix Factorization

In [1]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

# データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

In [1]:
# Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [2]:
# 因子数
factors = 10
# 評価数の閾値
minimum_num_rating = 0
# エポック数
n_epochs = 50
# alpha
alpha = 1.0

In [3]:
# 行列分解用に行列を作成する
filtered_movielens_train = movielens_train.groupby("movie_id").filter(
    lambda x: len(x["movie_id"]) >= minimum_num_rating
)

movielens_train_high_rating = filtered_movielens_train[filtered_movielens_train.rating >= 4]

# 行列のインデックスと映画/ユーザーを対応させる辞書を作成
unique_user_ids = sorted(movielens_train_high_rating.user_id.unique())
unique_movie_ids = sorted(movielens_train_high_rating.movie_id.unique())
user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
movie_id2index = dict(zip(unique_movie_ids, range(len(unique_movie_ids))))

In [4]:
from scipy.sparse import lil_matrix
# スパース行列を初期化して、各セルに値を入れていく
movielens_matrix = lil_matrix((len(unique_movie_ids), len(unique_user_ids)))
for i, row in movielens_train_high_rating.iterrows():
    user_index = user_id2index[row["user_id"]]
    movie_index = movie_id2index[row["movie_id"]]
    movielens_matrix[movie_index, user_index] = 1.0 * alpha # 今回、映画の評価値を0/1で２値化していますが、クリック数などのデータのときは、log(click数)などの変形も効果的です

In [ ]:
!pip install implicit==0.4.4

In [ ]:
# colab上で、implicitライブラリの実行時にエラーが出る場合は、「ランタイム→ランタイムのタイプを変更」でハードウェア アクセラレータとしてGPUを選択してください

In [6]:
import implicit

# モデルの初期化
model = implicit.als.AlternatingLeastSquares(
    factors=factors, iterations=n_epochs, calculate_training_loss=True, random_state=1
)

In [7]:
# 学習
model.fit(movielens_matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
from collections import defaultdict

# 推薦
recommendations = model.recommend_all(movielens_matrix.T)
pred_user2items = defaultdict(list)
for user_id, user_index in user_id2index.items():
    movie_indexes = recommendations[user_index, :]
    for movie_index in movie_indexes:
        movie_id = unique_movie_ids[movie_index]
        pred_user2items[user_id].append(movie_id)
pred_user2items

  0%|          | 0/997 [00:00<?, ?it/s]

defaultdict(list,
            {1: [457, 150, 380, 590, 110, 595, 593, 318, 349, 587],
             2: [1196, 1, 589, 593, 780, 1198, 32, 480, 2571, 1240],
             3: [4993, 5952, 4226, 3578, 2571, 356, 2959, 4306, 318, 2858],
             4: [457, 356, 380, 593, 377, 318, 349, 539, 597, 296],
             5: [296, 260, 1193, 50, 318, 36, 1213, 25, 1208, 2858],
             6: [1210, 1240, 589, 1214, 541, 2858, 1200, 2762, 593, 1036],
             7: [858, 750, 1193, 111, 924, 541, 1221, 1208, 1214, 1213],
             8: [1210, 4993, 260, 2028, 5952, 2716, 1704, 293, 1610, 5445],
             9: [2997, 1089, 50, 296, 1136, 2959, 223, 608, 778, 1206],
             10: [593, 318, 608, 2858, 25, 296, 1247, 50, 2396, 1704],
             11: [589, 1036, 1214, 1200, 780, 1, 541, 1197, 1356, 110],
             12: [2571, 260, 1210, 1196, 2028, 1704, 32, 4993, 1527, 2329],
             13: [1196, 1198, 2571, 260, 858, 593, 2762, 1240, 2858, 318],
             14: [1, 4993, 1073, 5952, 780

In [10]:
# user_id=2のユーザーが学習データで、4以上の評価を付けた映画一覧
movielens_train_high_rating[movielens_train_high_rating.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
8381,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0


In [11]:
# user_id=2に対するおすすめ(1196, 1, 589)
movies[movies.movie_id.isin([1196, 1, 589])]

,movie_id,title,genre,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat..."
583,589,Terminator 2: Judgment Day (1991),"[Action, Sci-Fi]","[action, sci-fi, dvd, seen more than once, tim..."
1171,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i..."


In [ ]:
# IMFでは、factorsやalphaの設定が予測精度に重要なため、値を変えてお試しください